#구글드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

Mounted at /gdrive/


#라이브러리들

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from numba import jit

#데이터 준비

In [ ]:
gp=pd.read_csv('/content/drive/MyDrive/프로젝트2/구매상품TR_reorder.csv',chunksize = 2000000)
gp_li = list(gp)
order = pd.concat(gp_li)



customer = pd.read_csv('/content/drive/MyDrive/프로젝트2/고객데모14.csv')



goods =  pd.read_excel('/content/drive/MyDrive/프로젝트2/상품분류3.xlsx')

#Class

In [ ]:
class project():
  def __init__(self,customer, goods, order):
    self.goods = goods
    self.customer = customer
    self.order = order
#-------------------------데이터프레임 ---------------------------------------------------------------------------
  def show_order(self):
    return self.order
  def show_customer(self):
    return self.customer
  def show_goods(self):
    return self.goods
#-------------------------  RFM ----------------------------------------------------------------------------
  def get_rfm_14_binary(self):
    temp = self.customer[['14_4분기_구매금액','14년도_영수증갯수','14년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    print(np.median(temp.frequency))
    print(np.median(temp.monetary))
    temp.recency = np.where(temp.recency != 0, 1,0)
    temp.frequency = np.where(temp.frequency < np.median(temp.frequency),0,1)
    temp.monetary = np.where(temp.monetary < np.median(temp.monetary),0,1)
    li = []
    for i in temp.values.tolist():
      if i not in li:
        li.append(i)
    res=self.get_cust_seg(temp,sorted(li,reverse = True))
    temp['cust_seg'] =res
    return temp
  def get_rfm_15_binary(self):
    temp = self.customer[['15_4분기_구매금액','15년도_영수증갯수','15년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    print(np.median(temp.frequency))
    print(np.median(temp.monetary))
    temp.recency = np.where(temp.recency != 0, 1,0)
    temp.frequency = np.where(temp.frequency < np.median(temp.frequency),0,1)
    temp.monetary = np.where(temp.monetary < np.median(temp.monetary),0,1)
    print(np.median(temp.frequency))
    li = []
    for i in temp.values.tolist():
      if i not in li:
        li.append(i)
    res=self.get_cust_seg(temp,sorted(li,reverse = True))
    temp['cust_seg'] =res
    return temp

  def get_rfm_14(self):
    temp = self.customer[['14_4분기_구매금액','14년도_영수증갯수','14년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    temp.recency = np.where(temp.recency != 0, 1,0)
    return temp

  def get_rfm_15(self):
    temp = self.customer[['15_4분기_구매금액','15년도_영수증갯수','15년도_구매금액']].copy()
    temp.columns = ['recency','frequency','monetary']
    temp.recency = np.where(temp.recency != 0, 1,0)
    return temp
  def get_cust_seg(self,temp,li):
    res = []
    for x in range(len(temp)):
      res.append(li.index(temp.iloc[x].values.tolist()))
    return res
#------------------------군집화 및 주성분분석 시각화 -----------------------------------------------------------
  def get_kmeans_clustering(self,rfm,n_cluster):
    scaler = MinMaxScaler()
    scaled_value=scaler.fit_transform(rfm.values)
    kmeans = KMeans(n_clusters = n_cluster)
    kmeans.fit(scaled_value)
    rfm['cluster']=kmeans.labels_
    return rfm
  def get_cluster_scatterplot(self,dataframe):
    pca = PCA(n_components = 2)
    temp=pca.fit_transform(dataframe.iloc[:,:-1])
    res=pd.DataFrame(temp)

    res['cluster']=dataframe.iloc[:,-1].values.tolist()
    sns.scatterplot(data = res,x = res.columns[0],y = res.columns[1],hue = res.columns[-1])
    plt.show()
    return pca.explained_variance_ratio_
#--------------------------------------------------------------------------------------------------
  def set_feature_customer(self,featurename,values):
    self.customer[featurename] = values
  def merge_goods_order(self):
    self.goods.중분류코드 = self.goods.중분류코드.astype('object')
    return pd.merge(self.order,self.goods,on = ['소분류코드','중분류코드','제휴사']).sort_values(['고객번호','구매일자','구매시간']).reset_index().drop('index',axis = 1)
  def train(self):
    df = self.merge_goods_order()
    return df[df['구매일자']<20151000]
  def test(self):
    df = self.merge_goods_order()
    return df[df['구매일자']>20151000]
  def get_shopping_list_middle(self,goods_order):
    li = []
    for x in goods_order['영수증번호'].unique():
      li.append(goods_order[goods_order['영수증번호']==x]['Unnamed: 5'].unique().tolist())
    return li
  @jit(nopython = False)
  def get_receipt_order(self,number,o):
    res=o[o['고객번호']== number]['영수증번호'].unique()
    return res
  @jit(nopython = False)
  def get_goods_name(self,receipt_number,o):
    res=o[o['영수증번호']== receipt_number]['소분류코드'].values
    return res
  
  @jit(nopython = False)
  def get_add_to_cart_order(self,array1,array2):
    sum = []
    for x in range(len(array1)):
      sum += [i for i in range(1,array2[x]+1)]
    return sum
  def get_customer_id_list(self):
    return sorted(self.customer['고객번호'].unique())
  @jit(nopython = False)
  def comparison(self,current,past):
    res = []
    if len(current) == 1:
      if current in past:
        res += [1]
      else :
        res += [0]
    else :
      res+=(np.in1d(current,past)*1).tolist()
    return res
  @jit(nopython = False)
  def cal_reordered(self):
    d = self.merge_goods_order()
    o= d[['고객번호','영수증번호','소분류코드']].copy()
    cust_list= self.get_customer_id_list()
    res = []
    for cust_id in cust_list[18000:]:
      print(cust_id)
      length=self.get_receipt_order(cust_id,o)
      for i in range(len(length)):
        if i != 0:
          past=self.get_goods_name(length[i-1],o)
          current=self.get_goods_name(length[i],o)
          res+=self.comparison(current,past)
        else :
          current=self.get_goods_name(length[i],o)
          for i in range(len(current)):
            res +=[0]
    pd.DataFrame(res).to_csv(f'/gdrive/MyDrive/프로젝트2/{cust_id}.csv',index = False)
    return res
  @jit(nopython = False)
  def get_cust_goods_kinds(self,dataframe):
    o=dataframe
    d = o.groupby(['고객번호'])['소분류코드'].value_counts().reset_index(name = 'cnt')
    res = []
    print(d)
    cust_list = self.get_customer_id_list()
    for x in cust_list:
      res.append(len(d[d['고객번호']== x]['소분류코드'].unique()))
    return res
  @jit(nopython = False)
  def get_cust_goods_kinds_mean(self,dataframe):
    o=dataframe
    d = o.groupby(['고객번호','영수증번호'])['소분류코드'].value_counts().reset_index(name = 'cnt')
    res = []
    print(d)
    cust_list = self.get_customer_id_list()
    for x in cust_list:
      res.append(d[d['고객번호'] == x]['cnt'].mean())
    return res        

#Class 생성

In [ ]:
p=project(customer,goods,order)

#항목수와 평균

In [ ]:
dataframe = p.train()
res1=p.get_cust_goods_kinds(dataframe)
res2=p.get_cust_goods_kinds_mean(dataframe)

KeyboardInterrupt: ignored

In [ ]:
dataframe = p.test()
res1=p.get_cust_goods_kinds(dataframe)
res2=p.get_cust_goods_kinds_mean(dataframe)

#Reorder

In [ ]:
res = p.cal_reordered()

<ipython-input-5-d82c86e57beb>:124: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "cal_reordered" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-5-d82c86e57beb> (126)

File "<ipython-input-5-d82c86e57beb>", line 126:
  def cal_reordered(self):
    d = self.merge_goods_order()
    ^

  @jit(nopython = False)
<ipython-input-5-d82c86e57beb>:124: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "cal_reordered" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-5-d82c86e57beb>", line 130:
  def cal_reordered(self):
      <source elided>
    res = []
    for cust_id in cust_list[18000:]:
    ^

  @jit(nopython = False)
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "cal_reordered" was compiled in object mod

18001


<ipython-input-5-d82c86e57beb>:96: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_receipt_order" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-5-d82c86e57beb> (98)

File "<ipython-input-5-d82c86e57beb>", line 98:
  def get_receipt_order(self,number,o):
    res=o[o['고객번호']== number]['영수증번호'].unique()
    ^

  @jit(nopython = False)
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "get_receipt_order" was compiled in object mode without forceobj=True.

File "<ipython-input-5-d82c86e57beb>", line 97:
  @jit(nopython = False)
  def get_receipt_order(self,number,o):
  ^

  warnings.warn(errors.NumbaWarning(warn_msg,
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is depreca

18002
18003
18004
18005
18006
18007
18008
18009
18010
18011
18012
18013
18014
18015
18016
18017
18018
18019
18020
18021
18022
18023
18024
18025
18026
18027
18028
18029
18030
18031
18032
18033
18034
18035
18036
18037
18038
18039
18040
18041
18042
18043
18044
18045
18046
18047
18048
18049
18050
18051
18052
18053
18054
18055
18056
18057
18058
18059
18060
18061
18062
18063
18064
18065
18066
18067
18068
18069
18070
18071
18072
18073
18074
18075
18076
18077
18078
18079
18080
18081
18082
18083
18084
18085
18086
18087
18088
18089
18090
18091
18092
18093
18094
18095
18096
18097
18098
18099
18100
18101
18102
18103
18104
18105
18106
18107
18108
18109
18110
18111
18112
18113
18114
18115
18116
18117
18118
18119
18120
18121
18122
18123
18124
18125
18126
18127
18128
18129
18130
18131
18132
18133
18134
18135
18136
18137
18138
18139
18140
18141
18142
18143
18144
18145
18146
18147
18148
18149
18150
18151
18152
18153
18154
18155
18156
18157
18158
18159
18160
18161
18162
18163
18164
18165
18166
18167
1816

In [ ]:
import os
os.listdir('/gdrive/MyDrive/프로젝트2/프로젝트')[-1]

'각고객의구매당평균항목수.csv'

#add_to_cart_order

In [ ]:
length=o.groupby('영수증번호')[['소분류코드']].count().reset_index()['소분류코드'].values
unique = sorted(o['영수증번호'].unique())
res=p.get_add_to_cart_order(unique,length)
print(len(res))
o=o.sort_values('영수증번호')
o['add_to_cart_order'] = res

<ipython-input-52-7ede92357c2a>:100: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_add_to_cart_order" failed type inference due to: Cannot infer the type of variable 'arg.self', have imprecise type: pyobject. 

File "unknown location", line 0:
<source missing, REPL/exec in use?>

  @jit(nopython = False)
<ipython-input-52-7ede92357c2a>:100: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "get_add_to_cart_order" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-52-7ede92357c2a>", line 103:
  def get_add_to_cart_order(self,array1,array2):
      <source elided>
    sum = []
    for x in range(len(array1)):
    ^

  @jit(nopython = False)
/usr/local/lib/python3.8/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "get_add_to_cart_order" was compiled in object mode without forceob

28593030


In [ ]:
o[['영수증번호','add_to_cart_order']].to_csv('/gdrive/MyDrive/프로젝트2/add_to_cart_order.csv',index = False)

#rfm

In [ ]:
df=p.get_rfm_15()
data=p.get_kmeans_clustering(df,8)

In [ ]:
df_new = df.copy()
df_new['cluster'] = data['cluster']
p.get_cluster_scatterplot(df_new)

In [ ]:
df_bin=p.get_rfm_15_binary()
df_bin.to_csv('15rfm.csv',index = False)

In [ ]:
p.get_rfm_14_binary()

270.0
8357765.0


,recency,frequency,monetary,cust_seg
0,1,1,1,0
1,1,1,1,0
2,1,1,0,1
3,1,0,1,2
4,1,1,0,1
...,...,...,...,...
19378,1,0,0,3
19379,1,0,0,3
19380,0,0,0,5
19381,0,0,0,5


#구매감소예측

In [ ]:
p.show_customer()['반기별_고객분류']

In [ ]:
p.set_feature_customer('구매증감지수',p.show_customer()['15_4분기_구매금액']/p.show_customer()['14_1분기_구매금액'])

In [ ]:
cus=p.show_customer()
sum=(cus['15년도_구매금액'].sum()/cus['14년도_구매금액'].sum())
d=cus[cus['구매증감지수'] < sum].groupby(['성별','연령대'])[['총구매금액']].mean().reset_index()

In [ ]:
d

In [ ]:
sns.barplot(data = d,x = '연령대',y = '총구매금액',hue = '성별')

In [ ]:
cus['15년도_구매금액'].sum()

347417316360

In [ ]:
cus['14년도_구매금액'].sum()

329601840581

In [ ]:
new = cus[['14_1분기_구매금액','14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액','15_2분기_구매금액', '15_3분기_구매금액', '15_4분기_구매금액']]
#계절성제거해야됨


target=((new['15_3분기_구매금액']-new['14_1분기_구매금액'])/new['14_1분기_구매금액'])*100
sum=((new['15_3분기_구매금액'].sum()-new['14_1분기_구매금액'].sum())/new['14_1분기_구매금액'].sum())*100

target2=((new['15_4분기_구매금액']-new['14_1분기_구매금액'])/new['14_1분기_구매금액'])*100
sum2=((new['15_4분기_구매금액'].sum()-new['14_1분기_구매금액'].sum())/new['14_1분기_구매금액'].sum())*100

In [ ]:
target=np.where(target == np.inf,0,target)
target2 = np.where(target2==np.inf,0,target2)

In [ ]:
cus['target_3'] = target.tolist()
cus['target'] = target2.tolist()

In [ ]:
cus['label_3'] = np.where(cus.target_3 < sum,0,1)
cus['label'] = np.where(cus.target < sum2,0,1)

In [ ]:
len(cus[cus['label'] == 0])/len(cus)

0.5857194448743744

In [ ]:
encoder = LabelEncoder()
sex=encoder.fit_transform(cus.성별)
age = encoder.fit_transform(cus.연령대)
address = encoder.fit_transform(cus.거주지역.astype('object'))

In [ ]:
cus['성별'] = sex
cus['연령대'] = age
cus['거주지역'] = address

In [ ]:
x_train=cus[['성별', '연령대', '거주지역','14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액', '15_3분기_구매금액','14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수','15년도_1분기_영수증갯수.1',
       '15년도_2분기_영수증갯수.1', '15년도_3분기_영수증갯수']]
y_train = cus[['label_3']]

In [ ]:
x_test=cus[['성별', '연령대', '거주지역','14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액', '15_3분기_구매금액','14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수','15년도_1분기_영수증갯수.1',
       '15년도_2분기_영수증갯수.1', '15년도_3분기_영수증갯수']]
y_test = cus[['label']]

In [ ]:

lr = LogisticRegression()
model = LGBMClassifier()
lr.fit(x_train,y_train)
model.fit(x_train,y_train)
preds=lr.predict(x_test)
prds2 = model.predict(x_test)
accuracy_score(preds,y_test)
accuracy_score(prds2,y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7210958056028478

In [ ]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Onion', 'Nutmeg', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)


frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

array([   0,    9,   22, 1381,   16,   12,   13,   17,   28, 1264,   63,
          8,   27,   20,   21,   27,   72])

In [ ]:
goods_order=p.merge_goods_order()

In [ ]:
goods_order

,제휴사,영수증번호,대분류코드_x,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액,대분류코드_y,중분류명,Unnamed: 5,소분류명
0,B,8664000,15,1504,B150401,17218,44,20140222,20,2420,가공식품,탄산음료,음료,사이다
1,B,8664007,15,1504,B150401,11303,44,20140222,13,2400,가공식품,탄산음료,음료,사이다
2,B,8919520,15,1504,B150401,13210,48,20150921,19,5250,가공식품,탄산음료,음료,사이다
3,B,6956422,15,1504,B150401,13823,12,20140813,18,2250,가공식품,탄산음료,음료,사이다
4,B,6957156,15,1504,B150401,18970,12,20140831,22,4900,가공식품,탄산음료,음료,사이다
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593025,B,7269975,14,1415,B141503,134,16,20141130,15,8900,가공식품,전통과자,과자류,온라인팥빙수재료
28593026,B,7946814,68,6802,B680209,18484,26,20141227,14,1000,패션잡화,여성양말,양말,여성발가락
28593027,B,7777110,7,702,B070203,5043,21,20151012,22,4510,가공식품,즉석구이안주,즉석/편의,즉석쥐포
28593028,B,7579178,7,703,B070305,4073,18,20140125,20,8900,가공식품,마른안주,즉석/편의,온라인 안주


In [ ]:
li = p.get_shopping_list_middle(goods_order)
te = TransactionEncoder()
te_ary = te.fit_transform(li)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
frequent_itemsets.to_csv('/gdrive/MyDrive/프로젝트2/arule.csv')

SyntaxError: ignored

In [ ]:
frequent_itemsets.to_csv('/gdrive/MyDrive/프로젝트2/프로젝트/연관성.csv',index = False) 

In [ ]:
df_cust=p.show_customer()[['고객번호', '성별', '연령대', '거주지역','14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액', '15_3분기_구매금액', '15_4분기_구매금액','14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수',
       '15년도_3분기_영수증갯수', '15년도_4분기_영수증갯수']]

In [ ]:
df_cust['분기별_구매금액_평균']=df_cust[['14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액','15_3분기_구매금액']].T.mean()


<ipython-input-17-ad030cb6aa16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cust['분기별_구매금액_평균']=df_cust[['14_1분기_구매금액',


In [ ]:
df_cust['분기별_영수증갯수_평균']=df_cust[['14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수','15년도_3분기_영수증갯수']].T.mean()

<ipython-input-19-539e98e995d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cust['분기별_영수증갯수_평균']=df_cust[['14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',


In [ ]:
df_cust['label'] = np.where(df_cust['15_3분기_구매금액'] != 0,1,0)

<ipython-input-31-617c53bd338b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cust['label'] = np.where(df_cust['15_3분기_구매금액'] != 0,1,0)


In [ ]:
df_cust.columns

Index(['고객번호', '성별', '연령대', '거주지역', '14_1분기_구매금액', '14_2분기_구매금액',
       '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액', '15_2분기_구매금액',
       '15_3분기_구매금액', '15_4분기_구매금액', '14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수',
       '15년도_3분기_영수증갯수', '15년도_4분기_영수증갯수', '분기별_구매금액_평균', '분기별_영수증갯수_평균',
       'label'],
      dtype='object')

In [ ]:
dataset = df_cust[['고객번호', '성별', '연령대', '거주지역','분기별_구매금액_평균','분기별_영수증갯수_평균','label']]

In [ ]:
dataset

,고객번호,성별,연령대,거주지역,분기별_구매금액_평균,분기별_영수증갯수_평균,label
0,1,M,60세이상,60.0,1.199001e+07,93.833333,1
1,2,M,60세이상,100.0,1.233949e+07,86.833333,1
2,3,M,60세이상,33.0,4.143415e+05,65.000000,1
3,4,F,60세이상,16.0,2.159426e+06,62.666667,1
4,5,M,60세이상,100.0,1.770135e+06,62.666667,0
...,...,...,...,...,...,...,...
19378,19379,F,20세~24세,69.0,5.977567e+05,35.666667,1
19379,19380,F,20세~24세,39.0,2.267750e+05,11.333333,1
19380,19381,F,40세~44세,460.0,4.661952e+05,25.833333,1
19381,19382,F,45세~49세,460.0,3.044565e+05,24.333333,1


In [ ]:
en = LabelEncoder()
sex=en.fit_transform(dataset['성별'])
age=en.fit_transform(dataset['연령대'])

In [ ]:
dataset['성별'] = sex
dataset['연령대'] = age

<ipython-input-36-80506ea1d8d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['성별'] = sex
<ipython-input-36-80506ea1d8d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['연령대'] = age


In [ ]:

x_train = dataset[['고객번호', '성별', '연령대', '거주지역', '분기별_구매금액_평균', '분기별_영수증갯수_평균']]
x_train.거주지역 = np.nan_to_num(x_train.거주지역,nan = 0)
x_train.거주지역 = x_train.거주지역.astype("int")
y_train = dataset['label']
dataset.label.value_counts()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


1    19305
0       78
Name: label, dtype: int64

In [ ]:
test = dataset.copy()

In [ ]:
test['분기별_구매금액_평균']=df_cust[['14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액','15_3분기_구매금액']].T.mean()
test['분기별_영수증갯수_평균']=df_cust[['14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수','15년도_3분기_영수증갯수']].T.mean()

In [ ]:
test['label'] = np.where(df_cust['15_4분기_구매금액'] != 0,1,0)

In [ ]:
test.거주지역 = np.nan_to_num(test.거주지역,nan = 0)
test.거주지역 = test.거주지역.astype('int')
x_test = test[['고객번호', '성별', '연령대', '거주지역', '분기별_구매금액_평균', '분기별_영수증갯수_평균']]
y_test = test['label']


In [ ]:
test.label.value_counts()

1    19319
0       64
Name: label, dtype: int64

In [ ]:
lr = LogisticRegression()
lr.fit(x_train,y_train)
preds=lr.predict(x_test)
f1_score(preds,y_test)

0.9983463386905069

In [ ]:
p=list(preds)
y = y_test.values.tolist()
id = []
cnt = 0
for i,j in zip(p,y):
  cnt +=1
  if j != i:
    print(cnt)
    id.append(cnt)

In [ ]:
dataset.iloc[id]

,고객번호,성별,연령대,거주지역,분기별_구매금액_평균,분기별_영수증갯수_평균,label
174,175,0,9,55.0,4.120923e+06,56.000000,1
460,461,0,9,460.0,1.129647e+06,59.166667,1
905,906,0,8,100.0,1.614656e+06,66.666667,1
1171,1172,0,8,16.0,1.171768e+07,91.666667,1
1208,1209,0,8,24.0,3.386207e+06,99.000000,1
...,...,...,...,...,...,...,...
19225,19226,0,5,460.0,2.629698e+06,44.166667,1
19229,19230,1,8,60.0,1.702333e+04,0.500000,0
19230,19231,0,5,0.0,0.000000e+00,0.000000,1
19240,19241,0,5,100.0,1.114680e+06,85.500000,1


In [ ]:
goods_=goods_order.sort_values(['구매일자','구매시간'])

In [ ]:
goods_order[goods_order['고객번호'] == 1]['영수증번호']

#train+test 합치기

In [ ]:
cust = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/cust+train_reorder.csv')

In [ ]:
test = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/고객_test.csv')
train =pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/고객_train.csv')

In [ ]:
len(p.train())

In [ ]:
train

,고객번호,구매상품수,첫-마지막,구매간격,구매건수,구매항목수,구매_평균항목수
0,1,979.0,622.0,2.990385,631.0,245,1.551506
1,2,997.0,634.0,2.486275,602.0,136,1.656146
2,3,1098.0,627.0,1.756303,443.0,155,2.478555
3,4,823.0,635.0,2.860360,455.0,234,1.808791
4,5,630.0,504.0,2.507463,376.0,68,1.675532
...,...,...,...,...,...,...,...
19378,19379,470.0,373.0,2.107345,298.0,114,1.577181
19379,19380,172.0,293.0,3.805195,94.0,61,1.829787
19380,19381,601.0,270.0,1.901408,283.0,240,2.123675
19381,19382,629.0,250.0,1.760563,280.0,247,2.246429


In [ ]:
test_size = len(p.test())
train_size= len(p.train())

In [ ]:
df = train.copy()
df['구매상품수'] = test['구매상품수']+train['구매상품수']
df['구매간격']= (test['구매간격']*test_size+train['구매간격']*train_size)/(test_size+train_size)
df['구매건수']= test['구매건수']+train['구매건수']
df['구매항목수']= test['구매항목수']+train['구매항목수']
df['구매_평균항목수']= (test['구매_평균항목수']*test_size + train['구매_평균항목수']*train_size)/(test_size+train_size)

In [ ]:
cust.columns

Index(['고객번호', '성별', '연령대', '거주지역', '시구', '시군구', '총구매금액', '14_1분기_구매금액',
       '14_2분기_구매금액', '14_3분기_구매금액', '14_4분기_구매금액', '15_1분기_구매금액',
       '15_2분기_구매금액', '15_3분기_구매금액', '15_4분기_구매금액', '14_1반기_구매금액',
       '14_2반기_구매금액', '15_1반기_구매금액', '15_2반기_구매금액', '14년도_구매금액', '15년도_구매금액',
       '14년도_반기별_증감율', '15년도_반기별_증감율', '14년도_후반기_15년도_초반기_증감율', '년도별_증감율',
       '영수증_구매일자_모음', '영수증_구매금액_모음', '첫구매일자및시간', '첫구매물품대분류명', '첫구매물품소분류명',
       '최다구매물품명', '최다구매물품수량', '최고금액구매물품명', '최고금액구매물품수량', '최고금액구매물품가격',
       '최고금액구매물품구매금액', '반기별_고객분류', '14년도_영수증갯수', '15년도_영수증갯수',
       '14년도_1분기_영수증갯수', '14년도_2분기_영수증갯수', '15년도_1반기_영수증갯수', '15년도_2반기_영수증갯수',
       '14년도_3분기_영수증갯수', '14년도_4분기_영수증갯수', '15년도_1분기_영수증갯수', '15년도_2분기_영수증갯수',
       '15년도_3분기_영수증갯수', '15년도_4분기_영수증갯수', '14년도_1반기_영수증갯수', '14년도_2반기_영수증갯수',
       '상품재구매비율', 'reorder_train'],
      dtype='object')

In [ ]:
d=pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/cust_reorder_predict.csv')

In [ ]:
d['label'] =cust.reorder_train

In [ ]:
df['label'] = cust.상품재구매비율

#예측 모델링

In [ ]:
!pip install mljar-supervised


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7

In [ ]:
order=p.show_order()
t=len(order[order['reorder'] == 1])/order['reorder'].count()
o = order[order['구매일자'] < 20151000]
t2 = t=len(o[o['reorder'] == 1])/o['reorder'].count()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
train=sc.fit_transform(d[['고객번호', '구매상품수', '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수']])
train_target = d['label']
train_target = np.where(train_target.values < t2,0,1)
test = sc.fit_transform(df[['고객번호', '구매상품수', '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수']])
test_target = df['label']
test_target = np.where(test_target.values < t,0,1)

In [ ]:
from supervised.automl import AutoML
automl = AutoML(mode = 'Compete', eval_metric='f1', ml_task='binary_classification', n_jobs=-1)

In [ ]:
automl.fit(train,train_target)

Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is binary_classification with evaluation metric f1
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree f1 0.715256 trained in 2.91 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree f1 0.713472 trained in 6.72 seconds
2_DecisionTree f1 0.724627 trained in 7.77 seconds
3_DecisionTree f1 0.724627 tr

/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `fe

5_Default_Xgboost f1 0.729744 trained in 34.5 seconds
6_Default_CatBoost f1 0.739585 trained in 26.42 seconds
7_Default_NeuralNetwork f1 0.730152 trained in 94.38 seconds
8_Default_RandomForest f1 0.731658 trained in 29.49 seconds
9_Default_ExtraTrees f1 0.691936 trained in 34.68 seconds
10_Default_NearestNeighbors f1 0.677733 trained in 7.87 seconds
* Step not_so_random will try to check up to 61 models
[1]	train's f1: 0	validation's f1: 0
[2]	train's f1: 0	validation's f1: 0
[3]	train's f1: 0	validation's f1: 0
[4]	train's f1: 0	validation's f1: 0
[5]	train's f1: 0.529572	validation's f1: 0.529825
[6]	train's f1: 0.524714	validation's f1: 0.502674
[7]	train's f1: 0.59904	validation's f1: 0.585326
[8]	train's f1: 0.595483	validation's f1: 0.571665
[9]	train's f1: 0.647174	validation's f1: 0.631415
[10]	train's f1: 0.65853	validation's f1: 0.646388
[11]	train's f1: 0.652857	validation's f1: 0.646388
[12]	train's f1: 0.675885	validation's f1: 0.664196
[13]	train's f1: 0.684536	validatio

/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `fe

11_Xgboost f1 0.730393 trained in 34.04 seconds
29_CatBoost f1 0.735967 trained in 31.09 seconds
38_RandomForest f1 0.721279 trained in 25.86 seconds
47_ExtraTrees f1 0.673442 trained in 28.23 seconds
56_NeuralNetwork f1 0.733051 trained in 92.95 seconds
65_NearestNeighbors f1 0.682901 trained in 13.02 seconds
[1]	train's f1: 0.39189	validation's f1: 0.348697
[2]	train's f1: 0.611632	validation's f1: 0.557404
[3]	train's f1: 0.646376	validation's f1: 0.615265
[4]	train's f1: 0.663736	validation's f1: 0.630617
[5]	train's f1: 0.691213	validation's f1: 0.643815
[6]	train's f1: 0.69493	validation's f1: 0.652461
[7]	train's f1: 0.709279	validation's f1: 0.663295
[8]	train's f1: 0.707661	validation's f1: 0.664265
[9]	train's f1: 0.717153	validation's f1: 0.673804
[10]	train's f1: 0.722524	validation's f1: 0.682171
[11]	train's f1: 0.726981	validation's f1: 0.68864
[12]	train's f1: 0.730807	validation's f1: 0.691771
[13]	train's f1: 0.734828	validation's f1: 0.688502
[14]	train's f1: 0.74002

...

/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `fe

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[119]	train's f1: 0.796846	validation's f1: 0.718729
[120]	train's f1: 0.797193	validation's f1: 0.717406
[121]	train's f1: 0.79725	validation's f1: 0.716931
[122]	train's f1: 0.797424	validation's f1: 0.717406
[123]	train's f1: 0.797685	validation's f1: 0.717406
[124]	train's f1: 0.798322	validation's f1: 0.716556
[125]	train's f1: 0.799132	validation's f1: 0.716656
[126]	train's f1: 0.799248	validation's f1: 0.715328
[127]	train's f1: 0.799797	validation's f1: 0.715328
[128]	train's f1: 0.799653	validation's f1: 0.715803
[129]	train's f1: 0.79974	validation's f1: 0.716656
[130]	train's f1: 0.799971	validation's f1: 0.718357
[1]	train's f1: 0	validation's f1: 0
[2]	train's f1: 0	validation's f1: 0
[3]	train's f1: 0	validation's f1: 0
[4]	train's f1: 0.0044724	validation's f1: 0.00503145
[5]	train's f1: 0.478079	validation's f1: 0.486782
[6]	train's f1: 0.537023	validation's f1: 0.525855
[7]	train's f1: 0.591687	validation's f1: 0.566639
[8]	train's 

/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
/usr/local/lib/python3.8/dist-packages/xgboost/training.py:39: UserWarning: `fe

14_Xgboost_Stacked f1 0.741305 trained in 51.99 seconds
* Step ensemble_stacked will try to check up to 1 model
Ensemble_Stacked f1 0.749027 trained in 48.36 seconds
AutoML fit time: 3675.81 seconds
AutoML best model: Ensemble_Stacked


AutoML(eval_metric='f1', ml_task='binary_classification', mode='Compete')

In [ ]:
lgbm = LGBMClassifier()
lgbm.fit(train,train_target)

LGBMClassifier()

In [ ]:
preds=lgbm.predict(test)

In [ ]:
f1_score(preds,test_target)

0.759929792628226

In [ ]:
!zip -r /content/drive/MyDrive/Automl.zip /content/AutoML_1

  adding: content/AutoML_1/ (stored 0%)
  adding: content/AutoML_1/folds/ (stored 0%)
  adding: content/AutoML_1/folds/fold_8_validation_indices.npy (deflated 79%)
  adding: content/AutoML_1/folds/fold_8_train_indices.npy (deflated 81%)
  adding: content/AutoML_1/folds/fold_0_train_indices.npy (deflated 81%)
  adding: content/AutoML_1/folds/fold_6_validation_indices.npy (deflated 80%)
  adding: content/AutoML_1/folds/fold_2_train_indices.npy (deflated 81%)
  adding: content/AutoML_1/folds/fold_5_validation_indices.npy (deflated 80%)
  adding: content/AutoML_1/folds/fold_5_train_indices.npy (deflated 81%)
  adding: content/AutoML_1/folds/fold_9_train_indices.npy (deflated 81%)
  adding: content/AutoML_1/folds/fold_3_train_indices.npy (deflated 81%)
  adding: content/AutoML_1/folds/fold_3_validation_indices.npy (deflated 80%)
  adding: content/AutoML_1/folds/fold_1_validation_indices.npy (deflated 80%)
  adding: content/AutoML_1/folds/fold_7_validation_indices.npy (deflated 80%)
  adding

#Catboost
- Catboost와 XGBoost의 공통점
level wise (트리에서 옆으로 확장) 방식의 부스팅
 
- Catboost와 XGBoost의 차이점 
Catboost 는 학습데이터의 일부만으로 잔차계산을 한 뒤, 이 결과로 모델을 다시 만들게 된다. Ordered Boosting - 학습 데이터를 선택할때 특정 순서를 가지고 선택

- Catboost의 특징
Cat, Category 즉 범주형 변수가 많은 데이터를 학습할 때 성능이 좋은 것으로 알려져있다.
- Response encoding과 Categorical Feature
Response Encoding - Catboost는 학습데이터 중 일부를 선택하여 학습시키는데, 이 때 일부를 선택하는 'order'은 시간 순서에 따른다. 만약 데이터에 시계열 데이터가 없다면 임의로 시간을 정하여 순서를 정한다(Random Permutation). 데이터의 시간 순으로 나열했을 때 과거 데이터만으로 response encoding을 진행하게 되는 것이다.
Categorical Feature Combination- 말 그대로 두 범주형 변수를 합쳐주는 것인데, label값을 기준으로 split을 할 때 두 범주형 변수가 서로 대체 가능하다면 이 둘을 합쳐준다.

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━ 34.3/76.6 MB 30.7 MB/s eta 0:00:02
ERROR: Operation cancelled by user


In [ ]:
d = pd.read_csv('/content/drive/MyDrive/프로젝트2/train_test/고객예측_train.csv')
df = pd.read_csv('/content/drive/MyDrive/프로젝트2/train_test/고객예측_test.csv')
cust = pd.read_csv('/content/drive/MyDrive/프로젝트2/train_test/cust.csv')

FileNotFoundError: ignored

In [ ]:
en=LabelEncoder()
sex=en.fit_transform(cust.성별)
age = en.fit_transform(cust.연령대)

In [ ]:
train.drop('Unnamed: 0',axis = 1,inplace = True)

In [ ]:
test.drop('Unnamed: 0',axis = 1,inplace = True)

In [ ]:
order=p.show_order()
t=len(order[order['reorder'] == 1])/order['reorder'].count()
o = order[order['구매일자'] < 20151000]
t2 = t=len(o[o['reorder'] == 1])/o['reorder'].count()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
train=sc.fit_transform(d[['고객번호', '구매상품수', '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수']])
train_target = d['label']
train_target = np.where(train_target.values < t2,0,1)
test = sc.fit_transform(df[['고객번호', '구매상품수', '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수']])
test_target = df['label']
test_target = np.where(test_target.values < t,0,1)

In [ ]:
cust.성별 = sex
cust.연령대 = age

In [ ]:
train=pd.DataFrame(train,columns = ['고객번호', '구매상품수', '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수'])
train=pd.concat([train,cust['성별'],cust['연령대'],cust['거주지역']],axis =1)

In [ ]:
test=pd.DataFrame(test,columns = ['고객번호', '구매상품수', '첫-마지막', '구매간격', '구매건수', '구매항목수', '구매_평균항목수'])
test=pd.concat([test,cust['성별'],cust['연령대'],cust['거주지역']],axis =1)

In [ ]:
from catboost import CatBoostClassifier, Pool

test_data = catboost_pool = Pool(test, test_target)

model = CatBoostClassifier(depth=7,
                           learning_rate=0.1,
                           loss_function='Logloss',
                           eval_metric = 'F1',
                           rsm = 1.0,
                           verbose=True)
# train the model
model.fit(train, train_target)
# make the prediction using the resulting model
preds_class = model.predict(test_data)
preds_proba = model.predict_proba(test_data)
print("class = ", preds_class)
print("proba = ", preds_proba)

In [ ]:
'''
- **n_jobs**: -1
- **learning_rate**: 0.1
- **depth**: 7
- **rsm**: 1.0
- **loss_function**: Logloss
- **eval_metric**: F1
- **explain_level**: 0
'''

In [ ]:
import matplotlib.pyplot as plt
model.feature_names_
plt.bar(['cust id','total ordered goods','days between first and last order','days between ordres','number of orders','kinds of ordered goods','mean of goods in order','sex','age','address'],model.feature_importances_,ec = 'skyblue',fc= 'lightgreen')
plt.xticks(rotation  = 90)

In [ ]:
f1_score(preds_class, test_target)

In [ ]:
accuracy_score(preds_class, test_target)

In [ ]:
confusion_matrix(preds_class,test_target)

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626479 sha256=f31ca4e4f1be017730dc5de51003cd6ef191fb1fa1fa505cd6626eaebaedafa4
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


#knn basic


In [ ]:
cluster = pd.read_csv('/gdrive/MyDrive/프로젝트2/train_test/고객군집f.csv')
cust_id=cluster[cluster['cluster2']==1]['고객번호'].values

In [ ]:
import surprise
gp = pd.read_csv('/gdrive/MyDrive/프로젝트2/구매상품TR_reorder.csv',chunksize = 2000000)
gp_li = list(gp)
df=pd.concat(gp_li)

reorder_rate=df.groupby(['고객번호','소분류명'])[['reorder']].sum()
total_ordered_goods=df.groupby(['고객번호','소분류명'])[['reorder']].count().reset_index()
o=reorder_rate.reset_index()
o['rate'] = o['reorder']/total_ordered_goods['reorder']


en = LabelEncoder()
goods_name=en.fit_transform(o['소분류명'])

o['소분류명_encoded'] = goods_name
bins =pd.cut(o['rate'],5,labels = [1,2,3,4,5])
o['ratings'] = bins

o.columns = ['user_id','소분류명','reorder','rate','item_id','raiting']
o.info()
p = o[['user_id','item_id','raiting']].astype('int16')
p=p.query('user_id in @cust_id')
reader = surprise.dataset.Reader(rating_scale = (0.5,5.0))
data=surprise.dataset.Dataset.load_from_df(p,reader)
trainset = data.build_full_trainset()

sim_options = {'name': 'cosine'}
algo = surprise.KNNBasic(sim_options=sim_options)
try :
  algo.fit(trainset)
except:
  print(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134917 entries, 0 to 6134916
Data columns (total 6 columns):
 #   Column   Dtype   
---  ------   -----   
 0   user_id  int64   
 1   소분류명     object  
 2   reorder  int64   
 3   rate     float64 
 4   item_id  int64   
 5   raiting  category
dtypes: category(1), float64(1), int64(3), object(1)
memory usage: 239.9+ MB
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
len(p.item_id.unique())

3415

In [ ]:
b=algo.predict(uid = 288,iid = 111)

In [ ]:
@jit (nopython = False)
def x(uid,iid):
  user_id = []
  item_id = []
  score = []
  for i in uid:
    for j in iid:
      a = algo.predict(i,j)
      user_id.append(a[0])
      item_id.append(a[1])
      score.append(a[3])
    print(i)
  df_rating=pd.DataFrame(user_id,columns = ['user_id'])
  df_rating['item_id'] = item_id
  df_rating['score'] = score
  return df_rating

In [ ]:
res=x(cust_id,p.item_id.unique())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5091
5092
5093
5095
5098
5099
5100
5103
5105
5108
5109
5110
5111
5115
5117
5119
5122
5123
5126
5127
5128
5130
5132
5135
5138
5140
5144
5145
5146
5152
5153
5154
5156
5157
5160
5165
5166
5172
5173
5176
5179
5180
5183
5185
5187
5189
5191
5192
5193
5197
5200
5201
5206
5209
5217
5220
5223
5228
5229
5234
5239
5240
5242
5246
5253
5255
5260
5261
5264
5277
5278
5279
5280
5282
5283
5286
5287
5289
5292
5296
5298
5303
5307
5311
5312
5314
5317
5320
5322
5323
5330
5332
5335
5336
5339
5341
5346
5347
5348
5353
5354
5356
5359
5367
5375
5376
5378
5379
5383
5387
5390
5391
5395
5397
5398
5400
5403
5405
5406
5407
5414
5417
5418
5421
5423
5424
5425
5428
5430
5433
5438
5441
5444
5450
5451
5458
5468
5469
5471
5473
5474
5480
5481
5487
5488
5492
5495
5497
5500
5502
5505
5506
5507
5510
5517
5523
5527
5530
5533
5535
5537
5541
5543
5549
5550
5551
5553
5559
5561
5565
5571
5572
5573
5577
5581
5585
5586
5591
5593
5600
5609
5611
5616
5618
5619
5633
5635
5636
5638
5640
5641
5642
5645

In [ ]:
res.to_csv('/gdrive/MyDrive/프로젝트2/rating.csv',index = False)

In [ ]:
df_list = []
for i in res.user_id.unique():
  df_list.append(res[res['user_id'] == i].sort_values('score',ascending = False)[:5])


In [ ]:
temp=pd.concat(df_list,axis = 0)

In [ ]:
temp=temp.reset_index()

In [ ]:
temp.drop('index',axis = 1,inplace = True)

In [ ]:
pd.merge(temp,o,left_index =True, right_index = True).to_csv('top5knn+item_name.csv',index = False)